# Rotten Tomatoes TV Shows Analysis

This notebook scrapes TV show data from Rotten Tomatoes and performs comprehensive analysis including:
- Network distribution and performance comparison
- Genre popularity and ratings analysis  
- Language distribution insights
- Critics vs audience score comparisons

In [ ]:
#import all necessary libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

In [ ]:
url = "https://www.rottentomatoes.com/browse/tv_series_browse/?page=4"

In [ ]:
def scrap_movies(url):

#Step 1: Extracting main page content 
    data = requests.get(url)
    soup = BeautifulSoup(data.text, 'lxml')
    series_containers = soup.find_all('div', class_='flex-container')

    # checking number of TV shows
    print(f"There are {len(series_containers)} TV Shows to process")
    
    # creating empty lists
    all_shows = []
    latest_episodes = []
    cover_images = []
    cleaned_urls = []

    # looping through all TV shows
    for show in series_containers:  

        # extracting all show names (Element 1)
        show_data = show.find("span", class_="p--small").get_text(strip=True)
        all_shows.append(show_data)
    
        # extracting latest episode date (Element 2)
        latest_episode_tag = show.find(class_="smaller")
        if latest_episode_tag:  # Check if element exists
            episode_text = latest_episode_tag.get_text(strip=True)
            episode_date = episode_text.replace("Latest Episode: ", "").strip()
            latest_episodes.append(episode_date)
        else:
            latest_episodes.append("Missing")

        # extracting all images (Element 3)
        image = show.find(class_="posterImage").get("src")
        cover_images.append(image)

    # looping through all extracted show names
    for s in all_shows:

        # preparing URLs using extracted show names
        clean_url = s.lower().replace("&", "and")          # replacing & with 'and'
        clean_url = re.sub(r"[^\w\s-]", "", clean_url)     # removing all special characters at once
        clean_url = re.sub(r"[\s_-]+", "_", clean_url)     # replacing multiple spaces/underscores with single _
        cleaned_urls.append(clean_url)

# Step 2: Extracting individual pages' content
    all_critics_scores = []
    all_audience_scores = []
    all_synopsis = []
    all_networks = []
    all_genres = []
    all_ratings = []
    all_languages = []
    all_release_dates = []

    
    valid_ratings = ["TV-Y", "TV-Y7", "TV-G", "TV-PG", "TV-14", "TV-MA"]
    valid_genres = ["Action", "Adventure", "Animation", "Anime", "Biography", "Comedy", "Crime", "Documentary", "Drama", "Entertainment",
                "Faith & Spirituality", "Fantasy", "Game Show", "LGBTQ+", "Health & Wellness", "History", "Holiday", "Horror", "House & Garden", 
                "Kids & Family", "Music", "Musical", "Mystery & Thriller", "Nature", "News", "Reality", "Romance", "Sci-Fi", "Short", "Soap", 
                "Special Interest", "Sports", "Stand-Up", "Talk Show", "Travel", "Variety", "War", "Western"]

    for show_url in cleaned_urls:
        # building the individual URLs
        full_url = f"https://www.rottentomatoes.com/tv/{show_url}"
        data3 = requests.get(full_url)
        time.sleep(1) # time delay
        soup3 = BeautifulSoup(data3.text, 'lxml')
        
        # extracting all elements
        final_elements = soup3.find_all('rt-text', attrs={'data-qa': 'item-value'})
        genre_elements = soup3.find_all('rt-link', attrs={'data-qa': 'item-value'})
        synopsis_element = soup3.find('rt-text', attrs={'data-qa': 'synopsis-value'})  
        critic_element = soup3.find_all('rt-text', attrs={'slot': 'criticsScore'})
        audience_element = soup3.find_all('rt-text', attrs={'slot': 'audienceScore'})

        # pulling synopsis (Element 4)
        synopsis = synopsis_element.get_text(strip=True) if synopsis_element else 'Missing'
    
        # pulling network, language, date, and ratings (Elements 5-8)  
        if final_elements:
            network = final_elements[0].get_text(strip=True)
            language = final_elements[-2].get_text(strip=True)
            date = final_elements[-1].get_text(strip=True)
            ratings = final_elements[1].get_text(strip=True)
        else:
            network = language = date = ratings = 'Missing'

        # rating validation
        rating = ratings if ratings in valid_ratings else "Missing"
        
        # pulling critic scores (Element 9)
        if critic_element:
            critic_score = critic_element[0].get_text(strip=True)
            critic_score = critic_score if critic_score else 'Missing' #also checking for instances where the element exists but no text
        else:
            critic_score = "Missing"

        # pulling audience scores (Element 10)
        if audience_element:    
            audience_score = audience_element[0].get_text(strip=True)
            audience_score = audience_score if audience_score else 'Missing' 
        else:
            audience_score = "Missing"
        
        # pulling genres and validating (Element 11)
        if genre_elements:
            show_genres = [g.strip() for g in "| ".join([elem.get_text(strip=True) for elem in genre_elements]).split('|')]
            genre_list = [g for g in show_genres if g in valid_genres]
            genre_string = ', '.join(genre_list) if genre_list else 'Missing'
        else:
            genre_string = 'Missing'

        #error handling for cases where URL has "2025" attached
        if critic_score == "Missing" and audience_score == "Missing":
            retry_url = f"https://www.rottentomatoes.com/tv/{show_url}_2025"
            retry_data = requests.get(retry_url)
            time.sleep(1) # time delay
            soup3 = BeautifulSoup(retry_data.text, 'lxml')  # Replace soup3 with new data
        
            # Re-extract scores from new page
            critic_element = soup3.find_all('rt-text', attrs={'slot': 'criticsScore'})
            audience_element = soup3.find_all('rt-text', attrs={'slot': 'audienceScore'})
        
            # Re-extract critic score
            if critic_element:
                critic_score = critic_element[0].get_text(strip=True)
                critic_score = critic_score if critic_score else 'Missing'
        
            # Re-extract audience score
            if audience_element:    
                audience_score = audience_element[0].get_text(strip=True)
                audience_score = audience_score if audience_score else 'Missing'
        
            # Re-extract other elements since we have a new soup3
            final_elements = soup3.find_all('rt-text', attrs={'data-qa': 'item-value'})
            genre_elements = soup3.find_all('rt-link', attrs={'data-qa': 'item-value'})
            synopsis_element = soup3.find('rt-text', attrs={'data-qa': 'synopsis-value'})
        
            # Re-extract synopsis
            synopsis = synopsis_element.get_text(strip=True) if synopsis_element else 'Missing'
        
            # Re-extract network, language, date, ratings
            if final_elements:
                network = final_elements[0].get_text(strip=True)
                language = final_elements[-2].get_text(strip=True)
                date = final_elements[-1].get_text(strip=True)
                ratings = final_elements[1].get_text(strip=True)
            else:
                network = language = date = ratings = 'Missing'
        
            rating = ratings if ratings in valid_ratings else "Missing"
        
            # Re-extract genres
            if genre_elements:
                show_genres = [g.strip() for g in "| ".join([elem.get_text(strip=True) for elem in genre_elements]).split('|')]
                genre_list = [g for g in show_genres if g in valid_genres]
                genre_string = ', '.join(genre_list) if genre_list else 'Missing'
            else:
                genre_string = 'Missing'
        
    # adding all data into their respective lists
        all_synopsis.append(synopsis)
        all_networks.append(network)
        all_languages.append(language)
        all_release_dates.append(date)
        all_ratings.append(rating)
        all_critics_scores.append(critic_score)
        all_audience_scores.append(audience_score)
        all_genres.append(genre_string)

    print("Creating DataFrame...")
    
    all_tv_shows = list(zip(all_shows, latest_episodes, all_critics_scores, all_audience_scores, cover_images, all_synopsis, 
                        all_networks, all_genres, all_ratings, all_languages, all_release_dates))

    columns = ["Show Name", "Latest Episode", "Critics Score", "Audience Score", "Cover Image", "Synopsis",
           "Network", "Genre", "Rating", "Language", "Release Date"]

    TV_Shows = pd.DataFrame(all_tv_shows, columns = columns)

    print(f"Scraping completed! Retrieved {len(TV_Shows)} TV Shows")
    
    return TV_Shows

In [ ]:
TV_Shows = scrap_movies(url)

In [ ]:
TV_Shows.head(5)

In [ ]:
TV_Shows.to_csv("TV_Shows.csv", index=False)

In [ ]:
# importing libraries for exploratory data analysis

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

import plotly #used to create interactive visualizations
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("TV_Shows.csv")

In [ ]:
df.info

In [ ]:
df.isnull().sum()

There are no missing values in this dataset

In [ ]:
TV_Shows.shape

In [ ]:
TV_Shows.columns

In [ ]:
TV_Shows.describe()

In [ ]:
# Check missing data across all columns
missing_data = (TV_Shows == 'Missing').sum()
missing_data

## Network Analysis

In [ ]:
unique_networks = TV_Shows['Network'].nunique()
network_frequency = TV_Shows['Network'].value_counts()

print(f"There are {unique_networks} unique networks")
print("")
print(network_frequency.head(6))

In [ ]:
print(f"Average shows per network: {network_frequency.mean():.1f}")
print(f"Most shows on single network: {network_frequency.max()}")

In [ ]:
# Categorize networks by size
major_networks = network_frequency[network_frequency >= 5]
midsized_networks = network_frequency[network_frequency >=3]
small_networks = network_frequency[network_frequency == 1]

print(f"Major networks (5+ shows): {len(major_networks)}")
print(f"Mid-sized networks(3+ shows): {len(midsized_networks)}")
print(f"Single-show networks: {len(small_networks)}")

## Ratings Analysis

In [ ]:
#top 5 ratings
unique_ratings = TV_Shows['Rating'].nunique()
ratings_frequency = TV_Shows['Rating'].value_counts()

print(f"There are {unique_ratings} unique ratings")
print("")
print(ratings_frequency.head(5))

In [ ]:
# Excluding 'Missing' from the result
ratings_clean = TV_Shows[TV_Shows['Rating'] != 'Missing']['Rating'].value_counts()

# Creating pie chart data
pie_labels = list(ratings_clean.index)
pie_values = list(ratings_clean.values)

# Creating pie chart
plt.figure(figsize=(6, 5))
plt.pie(pie_values, labels=pie_labels, autopct='%1.1f%%', startangle=180)
plt.title('Rating Distribution of TV Shows')
plt.show()

Most released TV shows, a high 76.3%, are rated TV-MA (mature content) which must mean that there is greater appetite for such shows.

## Language Analysis

In [ ]:
#top 5 languages
unique_language = TV_Shows['Language'].nunique()
language_frequency = TV_Shows['Language'].value_counts()

print(f"There are {unique_language} unique languages")
print("")
print(language_frequency)

In [ ]:
# Excluding 'Missing' from the result
language_clean = TV_Shows[TV_Shows['Language'] != 'Missing']['Language'].value_counts()

# Get top 4 languages
languages_5 = language_clean.head(5)

# Add remaining languages into one
others_count = language_clean.iloc[5:].sum()

# Creating pie chart data
pie_labels = list(languages_5.index) + ['Others']
pie_values = list(languages_5.values) + [others_count]

# Creating pie chart
plt.figure(figsize=(12, 8))
plt.pie(pie_values, labels=pie_labels, autopct='%1.1f%%', startangle=180)
plt.title('Language Distribution of TV Shows')
plt.show()

English content dominates the TV shows market, with 90.2% of shows released in English language. The content is spread across Australian, British, and general (probably American) English. Yet, The remaining 9.8% of shows are spread across other languages, showing emerging international markets.

### Critics Score Analysis

In [ ]:
critics = TV_Shows[TV_Shows['Critics Score'] != 'Missing']['Critics Score']
print(f"Shows with critics scores: {len(critics)}")

# removing % sign
critics_numeric = critics.str.replace('%', '').astype(int)

# Critics score statistics
print(f"Average: {critics_numeric.mean():.1f}%")
print(f"Median: {critics_numeric.median():.1f}%")
print(f"Highest: {critics_numeric.max()}%")
print(f"Lowest: {critics_numeric.min()}%")

### Audience Score Analysis

In [ ]:
audience = TV_Shows[TV_Shows['Audience Score'] != 'Missing']['Audience Score']
print(f"Shows with audience scores: {len(audience)}")

# removing % sign
audience_numeric = audience.str.replace('%', '').astype(int)

# Critics score statistics
print(f"Average: {audience_numeric.mean():.1f}%")
print(f"Median: {audience_numeric.median():.1f}%")
print(f"Highest: {audience_numeric.max()}%")
print(f"Lowest: {audience_numeric.min()}%")

### Release Date Analysis

In [ ]:
release_years = []

for date in TV_Shows['Release Date']:
    if date != 'Missing':
        parts = date.split()
        for part in parts:
            if len(part) == 4 and part.isdigit():
                year = int(part)
                if 1990 <= year <= 2025:  # reasonable range
                    release_years.append(year)
                    break

year_counts = pd.Series(release_years).value_counts().sort_index()
print("Shows by Release Year:")
print(year_counts)
print(f"\nYear with most releases: {year_counts.idxmax()} ({year_counts.max()} shows)")

In [ ]:
# working with top 5 networks
top_networks = TV_Shows['Network'].value_counts().head(6).index

networks = []
critics_scores = []
audience_scores = []

for network in top_networks:
    network_shows = TV_Shows[TV_Shows['Network'] == network]
    
    critics_data = network_shows[network_shows['Critics Score'] != 'Missing']['Critics Score']
    critics_avg = critics_data.str.replace('%', '').astype(int).mean() if len(critics_data) > 0 else 0
    
    audience_data = network_shows[network_shows['Audience Score'] != 'Missing']['Audience Score']
    audience_avg = audience_data.str.replace('%', '').astype(int).mean() if len(audience_data) > 0 else 0

    networks.append(network)
    critics_scores.append(critics_avg)
    audience_scores.append(audience_avg)

# Create bar chart
plt.figure(figsize=(10, 6))
x = range(len(networks))
plt.bar([i-0.2 for i in x], critics_scores, width=0.4, label='Critics', color='blue')
plt.bar([i+0.2 for i in x], audience_scores, width=0.4, label='Audience', color='red')

plt.xlabel('Network')
plt.ylabel('Average Score (%)')
plt.title('Critics vs Audience Scores by Network')
plt.xticks(x, networks)
plt.legend()
plt.show()

### Genre Analysis

In [ ]:
# Genre vs Critics Score
genre_critic_scores = []

for i in range(len(TV_Shows)):
    genre = TV_Shows.iloc[i]['Genre']
    critic_score = TV_Shows.iloc[i]['Critics Score']
    
    if genre != 'Missing' and critic_score != 'Missing':
        # Handle multiple genres (split by comma)
        genres = [g.strip() for g in genre.split(',')]
        score = int(critic_score.replace('%', ''))
        
        for g in genres:
            genre_critic_scores.append([g, score])

# Create DataFrame for easier analysis
genre_scores_df = pd.DataFrame(genre_critic_scores, columns=['Genre', 'Score'])

# Calculate average score per genre
genre_avg_critics = genre_scores_df.groupby('Genre')['Score'].mean().sort_values(ascending=False)

# Create bar chart
plt.figure(figsize=(12, 6))
plt.bar(range(len(genre_avg_critics)), genre_avg_critics.values, color='green')
plt.xlabel('Genre')
plt.ylabel('Average Critics Score (%)')
plt.title('Average Critics Scores by Genre')
plt.xticks(range(len(genre_avg_critics)), genre_avg_critics.index, rotation=45, ha='right')

plt.tight_layout()
plt.show()

Critics are more benevolent towards their ratings for Anime, offering an average of 100% compared to Romance with a rating below 80%

In [ ]:
# Genre vs Critics Score
genre_audience_scores = []

for i in range(len(TV_Shows)):
    genre = TV_Shows.iloc[i]['Genre']
    audience_score = TV_Shows.iloc[i]['Audience Score']
    
    if genre != 'Missing' and audience_score != 'Missing':
        # Handle multiple genres (split by comma)
        genres = [g.strip() for g in genre.split(',')]
        score = int(audience_score.replace('%', ''))
        
        for g in genres:
            genre_audience_scores.append([g, score])

# Create DataFrame for easier analysis
genre_score_df = pd.DataFrame(genre_audience_scores, columns=['Genre', 'Score'])

# Calculate average score per genre
genre_avg_aud = genre_score_df.groupby('Genre')['Score'].mean().sort_values(ascending=False)

# Create bar chart
plt.figure(figsize=(12, 6))
plt.bar(range(len(genre_avg_aud)), genre_avg_aud.values, color='lightblue')
plt.xlabel('Genre')
plt.ylabel('Average Audience Score (%)')
plt.title('Average Audience Scores by Genre')
plt.xticks(range(len(genre_avg_aud)), genre_avg_aud.index, rotation=45, ha='right')

plt.tight_layout()
plt.show()

The audience seems to be more favourable towards documentaries, which got the highest rating of 100%, but romance still falls on the low end of below 80%

In [ ]:
top_networks = TV_Shows['Network'].value_counts().head(8).index

network_best_genres = []

for network in top_networks:
    network_shows = TV_Shows[TV_Shows['Network'] == network]
    
    # Get genre-audience scores for this network only
    network_genre_scores = []
    
    for i in range(len(network_shows)):
        genre = network_shows.iloc[i]['Genre']
        audience_score = network_shows.iloc[i]['Audience Score']
        
        if genre != 'Missing' and audience_score != 'Missing':
            genres = [g.strip() for g in genre.split(',')]
            score = int(audience_score.replace('%', ''))
            
            for g in genres:
                network_genre_scores.append([g, score])
    
    # Find this network's highest rated genre
    if network_genre_scores:
        network_df = pd.DataFrame(network_genre_scores, columns=['Genre', 'Score'])
        network_genre_avg = network_df.groupby('Genre')['Score'].mean()
        
        best_genre = network_genre_avg.idxmax()
        best_score = network_genre_avg.max()
        network_best_genres.append([network, best_genre, best_score])

# Visualization
networks = [item[0] for item in network_best_genres]
scores = [item[2] for item in network_best_genres]
genres = [item[1] for item in network_best_genres]

plt.figure(figsize=(8, 6))
bars = plt.bar(networks, scores, color='skyblue')

for bar, genre, score in zip(bars, genres, scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{genre}\n{score:.1f}%', ha='center', va='bottom')

plt.title('Each Network\'s Best Performing Genre (Audience Score)')
plt.ylabel('Average Score (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
top_networks = TV_Shows['Network'].value_counts().head(8).index

network_best_genres = []

for network in top_networks:
    network_shows = TV_Shows[TV_Shows['Network'] == network]
    
    # Get genre-critic scores for this network only
    network_genre_scores = []
    
    for i in range(len(network_shows)):
        genre = network_shows.iloc[i]['Genre']
        critics_score = network_shows.iloc[i]['Critics Score']
        
        if genre != 'Missing' and critics_score != 'Missing':
            genres = [g.strip() for g in genre.split(',')]
            score = int(critics_score.replace('%', ''))
            
            for g in genres:
                network_genre_scores.append([g, score])
    
    # Find this network's highest rated genre
    if network_genre_scores:
        network_df = pd.DataFrame(network_genre_scores, columns=['Genre', 'Score'])
        network_genre_avg = network_df.groupby('Genre')['Score'].mean()
        
        best_genre = network_genre_avg.idxmax()
        best_score = network_genre_avg.max()
        network_best_genres.append([network, best_genre, best_score])

# Visualization
networks = [item[0] for item in network_best_genres]
scores = [item[2] for item in network_best_genres]
genres = [item[1] for item in network_best_genres]

plt.figure(figsize=(8, 6))
bars = plt.bar(networks, scores, color='orange')

for bar, genre, score in zip(bars, genres, scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{genre}\n{score:.1f}%', ha='center', va='bottom')

plt.title('Each Network\'s Best Performing Genre (Critics Score)')
plt.ylabel('Average Score (%)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Comedy and Action are more preferred by both audiences and critics

In [ ]:
# Categorize languages into English vs Other
english_variants = ['English', 'Australian English', 'British English']

# Separate shows by language category
english_critic_scores = []
other_critic_scores = []
english_audience_scores = []
other_audience_scores = []

for i in range(len(TV_Shows)):
    language = TV_Shows.iloc[i]['Language']
    critic_score = TV_Shows.iloc[i]['Critics Score']
    audience_score = TV_Shows.iloc[i]['Audience Score']
    
    if language != 'Missing':
        # Check if it's an English variant
        is_english = language in english_variants
        
        # Add critic scores
        if critic_score != 'Missing':
            score = int(critic_score.replace('%', ''))
            if is_english:
                english_critic_scores.append(score)
            else:
                other_critic_scores.append(score)
        
        # Add audience scores
        if audience_score != 'Missing':
            score = int(audience_score.replace('%', ''))
            if is_english:
                english_audience_scores.append(score)
            else:
                other_audience_scores.append(score)

# Calculate averages
english_critic_avg = sum(english_critic_scores) / len(english_critic_scores) if english_critic_scores else 0
other_critic_avg = sum(other_critic_scores) / len(other_critic_scores) if other_critic_scores else 0
english_audience_avg = sum(english_audience_scores) / len(english_audience_scores) if english_audience_scores else 0
other_audience_avg = sum(other_audience_scores) / len(other_audience_scores) if other_audience_scores else 0

# Visualization
categories = ['English Shows', 'International Languages']
critic_scores = [english_critic_avg, other_critic_avg]
audience_scores = [english_audience_avg, other_audience_avg]

plt.figure(figsize=(10, 10))
x = range(len(categories))
plt.bar([i-0.2 for i in x], critic_scores, width=0.4, label='Critics', color='blue')
plt.bar([i+0.2 for i in x], audience_scores, width=0.4, label='Audience', color='red')

plt.xlabel('Languages')
plt.ylabel('Average Score (%)')
plt.title('English vs International Languages - Critics vs Audience Scores')
plt.xticks(x, categories)

# Add value labels
for i, (critic, audience) in enumerate(zip(critic_scores, audience_scores)):
    plt.text(i-0.2, critic + 1, f'{critic:.1f}%', ha='center', va='bottom')
    plt.text(i+0.2, audience + 1, f'{audience:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

International languages like French and Korean score higher for both critics and audiences. We cannot use this inference directly in our recommendation because of the large gap in sample sizes. However, networks should consider some more diversity and release more international movies as both audiences and critics are well accepting of them

In [ ]:
# Genre across Networks (excluding Missing)
top_networks = TV_Shows[TV_Shows['Network'] != 'Missing']['Network'].value_counts().head(5).index

# Get top genres (excluding Missing)
all_genres = []
for genre_string in TV_Shows['Genre']:
    if genre_string != 'Missing':
        genres = [g.strip() for g in genre_string.split(',')]
        all_genres.extend(genres)

top_genres = pd.Series(all_genres).value_counts().head(5).index

# Create data for heatmap-style visualization (excluding Missing)
network_genre_data = {}
for network in top_networks:
    network_shows = TV_Shows[(TV_Shows['Network'] == network) & (TV_Shows['Genre'] != 'Missing')]
    
    network_genre_data[network] = {}
    for genre in top_genres:
        count = 0
        for i in range(len(network_shows)):
            show_genre = network_shows.iloc[i]['Genre']
            if genre in show_genre:
                count += 1
        network_genre_data[network][genre] = count

# Create visualization
plt.figure(figsize=(8, 6))

# Prepare data for grouped bar chart
genres_list = list(top_genres)
x = range(len(genres_list))
width = 0.15

colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold', 'plum']

for i, network in enumerate(top_networks):
    values = [network_genre_data[network][genre] for genre in genres_list]
    plt.bar([j + width*i for j in x], values, width, label=network, color=colors[i])

plt.xlabel('Genre')
plt.ylabel('Number of Shows')
plt.title('Genre Distribution Across Top 5 Networks')
plt.xticks([j + width*2 for j in x], genres_list, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

TV shows from Netflix are distributed across multiple genres, with a heavy focus on drama (64% of 25 Netflix shows)

### Database Storage

In [ ]:
import pymysql
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '12345',
    'database': 'practice'
}

In [ ]:
create_table_query = """
CREATE TABLE IF NOT EXISTS MOVIES (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Show_Name VARCHAR(255),
    Latest_Episode VARCHAR(50),
    Critics_Score VARCHAR(10),
    Audience_Score VARCHAR(10),
    Cover_Image TEXT,
    Synopsis TEXT,
    Network VARCHAR(100),
    Genre VARCHAR(200),
    Rating VARCHAR(20),
    Language VARCHAR(50),
    Release_Date VARCHAR(50),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
"""

cursor.execute(create_table_query)
mydb.commit()
print("MOVIES table created successfully!")

In [ ]:
insert_data_query = """
INSERT INTO MOVIES (Show_Name, Latest_Episode, Critics_Score, Audience_Score, Cover_Image, Synopsis, Network, Genre, Rating, Language, Release_Date)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

data_to_insert = []
for index, row in TV_Shows.iterrows():
    data_to_insert.append((
        row['Show Name'],
        row['Latest Episode'], 
        row['Critics Score'],
        row['Audience Score'],
        row['Cover Image'],
        row['Synopsis'],
        row['Network'],
        row['Genre'],
        row['Rating'],
        row['Language'],
        row['Release Date']
    ))

In [ ]:
cursor.executemany(insert_data_query, data_to_insert)
mydb.commit()
print(f" Inserted {len(data_to_insert)} TV shows into MOVIES table")

In [ ]:
read_back_df = pd.read_sql("SELECT * FROM MOVIES", mydb)
print(read_back_df.head(3))